In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [5]:
X=pd.read_csv('X.csv')

a1=list(X.Region.unique())
regions_dict=dict(zip(range(1,len(a1)+1),a1))
def my_date(dt):
    a= dt.split('-')
    return datetime.date(int(a[0]),int(a[1]),int(a[2]))
X.Date=X.Date.apply(my_date)
myday = list(X.tail(1).Date)[0]

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13363 entries, 0 to 13362
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                13363 non-null  object 
 1   Confirmed           13363 non-null  int64  
 2   Recovered           13363 non-null  int64  
 3   Deaths              13363 non-null  int64  
 4   Day_deaths          13363 non-null  int64  
 5   Day_confirmed       13363 non-null  int64  
 6   Day_recovered       13363 non-null  int64  
 7   Rt                  12206 non-null  float64
 8   Rt2                 12316 non-null  float64
 9   Rate_conf           13363 non-null  float64
 10  Rate_ill            13363 non-null  float64
 11  Region              13363 non-null  object 
 12  Remaining_ill       13363 non-null  int64  
 13  MA7_dayconfirmed    12847 non-null  float64
 14  MA14_dayconfirmed   12245 non-null  float64
 15  MA7_remaining_ill   12847 non-null  float64
 16  MA14

In [7]:
covid_rates = pd.DataFrame()
mask = (X.Date==myday)&(X.Region!='Россия')
today_cases = X[mask]
#covid_rates.reset_index(drop=True)

covid_rates['По количеству текущих больных'] = today_cases.sort_values(by=['Remaining_ill'],ascending = False).Region.reset_index(drop=True)
covid_rates['По количеству новых'] = today_cases.sort_values(by=['Day_confirmed'],ascending = False).Region.reset_index(drop=True)
covid_rates['По умершим за день'] = today_cases.sort_values(by=['Day_deaths'],ascending = False).Region.reset_index(drop=True)
covid_rates['По выздоровевшим за день'] = today_cases.sort_values(by=['Day_recovered'],ascending = False).Region.reset_index(drop=True)

fig =  ff.create_table(covid_rates.head(15))
fig.update_layout(
    
                  title_text ='Таблица рейтингов заболеваемости COVID-19 по регионам',
                  margin = {'t':100, 'b':100},
                  title_x = 0.5,
                  title_y= 0.95,
                  title_xanchor = "center",
                  title_yanchor = "top", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)
fig.show()




In [8]:
region_number = 5
regions_towatch = list(covid_rates['По количеству текущих больных'].head(region_number))
regions_towatch.append("Россия")
regions_towatch

['Москва',
 'Московская обл.',
 'Санкт-Петербург',
 'Челябинская обл.',
 'Саратовская обл.',
 'Россия']

In [9]:
mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))
#mask = (X.Region.isin(regions_towatch))

fig = px.line(X[mask], y="Day_confirmed", x="Date",
                             color="Region", line_group="Region",
              line_shape="spline", render_mode="notebook",labels={'Region':'Регион:'},
              color_discrete_map={
                "Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия': 'aquamarine'},
             title="График количества новых больных по 5 регионам с наибольшим количеством заболеваний")

fig.update_layout(
    annotations=[
            dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper")],
           
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)

fig.show()






In [10]:
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))

colors_dict = {"Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия':'aquamarine'}
fig = go.Figure()
for i in regions_towatch:
    mask = X['Region'] == i
        
    fig.add_trace(
        
        go.Bar(
        name = f'{i}',
        x=X[mask].Date,
        y=X[mask].Rt,
        text=round(X[mask].Rt,3),
        textposition='inside',
        marker_color= colors_dict.get(i),
        
        showlegend=True))




                  
fig.update_layout(
    annotations=[
        dict(
            x='2020-04-13',
            y=1.717,
            xref="x",
            yref="y",
            text="Введение<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
        dict(
            x='2020-06-09',
            y=0.942,
            xref="x",
            yref="y",
            text="Отмена<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
            dict(x=0.5,y=-0.15,
            showarrow=False,
            text="Данные : Яндекс <br> формула расчета из методических рекомендациях Роспотребнадзора.",
            xref="paper",
            yref="paper")],
                  title = "Коэффициент распространения (Rt) по 5 регионам ,<br> с максимальным количеством новых заболеваний ",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.9,
                  width = 990, height = 600,template = 'gridon',
    
    
                 # xaxis_title='Коэффициент рассчитан, исходя из данных оперативного штаба и <br>' + 
                               #' формулы расчета в методических рекомендациях Роспотребнадзора. ',
    yaxis_title = ' '
)


fig.show()


In [11]:
print('Все регионы по которым есть статистика:')
print('*'*40)
print(regions_dict.items())
print('*'*40)
print('Рассматриваем регионы: ')
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))


my_region = regions_dict[41]
print('Выбрали регион: ',my_region)


Все регионы по которым есть статистика:
****************************************
dict_items([(1, 'Адыгея'), (2, 'Алтай'), (3, 'Алтайский край'), (4, 'Амурская обл.'), (5, 'Архангельская обл.'), (6, 'Астраханская обл.'), (7, 'Башкортостан'), (8, 'Белгородская обл.'), (9, 'Брянская обл.'), (10, 'Бурятия'), (11, 'Владимирская обл.'), (12, 'Волгоградская обл.'), (13, 'Вологодская обл.'), (14, 'Воронежская обл.'), (15, 'Дагестан'), (16, 'Еврейская АО'), (17, 'Забайкальский край'), (18, 'Ивановская обл.'), (19, 'Ингушетия'), (20, 'Иркутская обл.'), (21, 'Кабардино-Балкария'), (22, 'Калининградская обл.'), (23, 'Калмыкия'), (24, 'Калужская обл.'), (25, 'Камчатский край'), (26, 'Карачаево-Черкессия'), (27, 'Карелия'), (28, 'Кемеровская обл.'), (29, 'Кировская обл.'), (30, 'Коми'), (31, 'Костромская обл.'), (32, 'Краснодарский край'), (33, 'Красноярский край'), (34, 'Крым'), (35, 'Курганская обл.'), (36, 'Курская обл.'), (37, 'Ленинградская обл.'), (38, 'Липецкая обл.'), (39, 'Магаданская обл.'

In [12]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region.isin(regions_towatch))
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region==my_region)

fig = px.bar(X[mask], x='Date', 
             y="Day_confirmed",color ='Day_confirmed',
             title=f'{my_region}: '+'график количества новых заболевших COVID-19 по датам',
              color_continuous_scale= 'jet',text = 'Day_confirmed',
             labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)
fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask]['Date'],
        y=X[mask]['MA7_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=1355,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=1572,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-90),
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper"),
        
                ],
    
    
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.15,legend_y = 0.95,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ' '
)


fig.show()




In [13]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region==my_region)
#Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
#Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = X[mask].Date,y=X[mask].Change_dayconf,text = X[mask]['Change_dayconf'],textposition = 'inside',
                
                
                 marker_color = X[mask]['Change_dayconf'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}}
                    )   
                )
fig.update_layout(
   annotations=[
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика новых заболевших COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Данные: Яндекс',
    yaxis_title = ' '
)

fig.show()






In [16]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region.isin(regions_towatch))

fig = px.bar(X[mask], x='Date', y="Remaining_ill",color ='Remaining_ill',
             title=f'{my_region}: '+'график количества текущих больных COVID-19 по датам',
              color_continuous_scale= 'tempo',text = 'Remaining_ill',
            labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)

fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA7_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=85824,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                                '<br>Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()





In [14]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region==my_region)
Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_remill,text = Y['Change_remill'],textposition = 'inside',
                
                #marker_color=Y['Change_remill'].apply(lambda d:'crimson' if d>0 else 'green'),
                marker_color = Y['Change_remill'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}})     
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-3877,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
#         dict(
#             x='2020-05-20',
#             y=-2816,
#             xref="x",
#             yref="y",
#             text="20 мая",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                               '<br>Данные : Яндекс',
    yaxis_title = ' '
)

fig.show()


In [15]:
mask = (X.Date>=datetime.date(2020,7,27))&(X.Region.isin(regions_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt.sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head()

[[1.101 1.083 1.072 1.051 1.018 1.013 1.002 1.009 1.007 1.    1.01  1.002
  1.004 1.005 1.002 1.007 1.007 1.007]
 [0.998 0.986 0.968 0.941 0.97  1.041 1.099 1.128 1.126 1.103 1.07  1.039
  1.021 0.98  0.967 0.964 0.95  0.968]
 [0.983 0.97  0.952 0.945 0.947 0.967 0.982 0.989 0.981 0.966 0.961 0.959
  0.976 0.987 0.987 0.98  0.973 0.967]
 [0.938 0.938 0.932 0.928 0.925 0.932 0.949 0.964 0.982 0.985 0.981 0.98
  0.977 0.991 0.994 0.998 1.013 1.006]
 [1.024 1.027 1.016 1.01  1.018 1.013 1.005 0.987 0.967 0.962 0.956 0.969
  0.987 0.997 1.024 1.024 1.021 1.013]
 [0.958 0.988 1.002 0.98  0.962 0.929 0.896 0.893 0.889 0.9   0.895 0.878
  0.861 0.854 0.867 0.897 0.943 0.939]]


Date,Region,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13
0,Москва,1.101,1.083,1.072,1.051,1.018,1.013,1.002,1.009,1.007,1.000,1.010,1.002,1.004,1.005,1.002,1.007,1.007,1.007
1,Московская обл.,0.998,0.986,0.968,0.941,0.970,1.041,1.099,1.128,1.126,1.103,1.070,1.039,1.021,0.980,0.967,0.964,0.950,0.968
2,Россия,0.983,0.970,0.952,0.945,0.947,0.967,0.982,0.989,0.981,0.966,0.961,0.959,0.976,0.987,0.987,0.980,0.973,0.967
3,Санкт-Петербург,0.938,0.938,0.932,0.928,0.925,0.932,0.949,0.964,0.982,0.985,0.981,0.980,0.977,0.991,0.994,0.998,1.013,1.006
4,Саратовская обл.,1.024,1.027,1.016,1.010,1.018,1.013,1.005,0.987,0.967,0.962,0.956,0.969,0.987,0.997,1.024,1.024,1.021,1.013


In [16]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print(x2)

[datetime.date(2020, 7, 27), datetime.date(2020, 7, 28), datetime.date(2020, 7, 29), datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13)]
['27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020']


In [17]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [18]:
mask = (X.Date>=datetime.date(2020,7,27))&(X.Region.isin(regions_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt2.sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head()

[[1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011
  1.011 1.011 1.011 1.011 1.011 1.011]
 [1.009 1.009 1.009 1.009 1.009 1.009 1.009 1.009 1.01  1.01  1.01  1.01
  1.01  1.01  1.01  1.01  1.01  1.009]
 [1.03  1.029 1.029 1.028 1.027 1.027 1.027 1.026 1.026 1.026 1.025 1.025
  1.025 1.024 1.024 1.024 1.024 1.023]
 [1.024 1.024 1.023 1.023 1.022 1.022 1.021 1.021 1.021 1.02  1.02  1.02
  1.02  1.02  1.02  1.019 1.019 1.019]
 [1.045 1.045 1.044 1.044 1.044 1.043 1.043 1.042 1.042 1.041 1.04  1.039
  1.039 1.038 1.038 1.038 1.038 1.038]
 [1.051 1.05  1.049 1.048 1.047 1.046 1.044 1.043 1.041 1.039 1.038 1.036
  1.035 1.033 1.032 1.03  1.03  1.029]]


Date,Region,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13
0,Москва,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011
1,Московская обл.,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.010,1.009
2,Россия,1.030,1.029,1.029,1.028,1.027,1.027,1.027,1.026,1.026,1.026,1.025,1.025,1.025,1.024,1.024,1.024,1.024,1.023
3,Санкт-Петербург,1.024,1.024,1.023,1.023,1.022,1.022,1.021,1.021,1.021,1.020,1.020,1.020,1.020,1.020,1.020,1.019,1.019,1.019
4,Саратовская обл.,1.045,1.045,1.044,1.044,1.044,1.043,1.043,1.042,1.042,1.041,1.040,1.039,1.039,1.038,1.038,1.038,1.038,1.038


In [19]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print(x2)

[datetime.date(2020, 7, 27), datetime.date(2020, 7, 28), datetime.date(2020, 7, 29), datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13)]
['27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020']


In [20]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt2 по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [21]:
mask = (X.Date>=datetime.date(2020,7,27))&(X.Region.isin(regions_towatch))

Z = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values
Z.head()

Date,Region,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13
0,Москва,694,674,671,678,695,690,664,693,691,687,684,686,691,689,694,694,689,692
1,Московская обл.,139,126,137,144,150,149,157,160,161,162,159,158,161,149,153,154,152,148
2,Россия,5635,5395,5475,5509,5482,5462,5427,5394,5159,5204,5267,5241,5212,5189,5118,4945,5102,5057
3,Санкт-Петербург,172,168,165,162,159,161,163,160,158,156,159,157,155,160,157,157,161,160
4,Саратовская обл.,95,96,98,99,98,96,95,94,93,94,90,94,95,96,95,94,96,95


In [22]:



x1=list(Z.columns[1:])
y=list(Z.Region)
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
print(x2)


['27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020']


In [23]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,
    
    
                  xaxis_title='Динамика новых заболевших по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()

In [24]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_conf'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_conf']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(15)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6521,2020-08-13,250303,193473,4622,11,692,787,1.007,1.011,27.573759,29.667345,Москва,52208,690.714286,688.500000,53014.714286,54670.785714,3,-106
6686,2020-08-13,65472,49066,1136,4,148,150,0.968,1.009,7.212495,8.677221,Московская обл.,15270,153.571429,155.214286,15348.714286,15462.642857,-4,-6
9337,2020-08-13,33525,24354,2203,15,160,96,1.006,1.019,3.693165,3.959586,Санкт-Петербург,6968,158.142857,158.785714,6763.000000,6507.714286,-1,49
7152,2020-08-13,25382,22197,421,3,130,242,1.113,1.019,2.796120,1.570651,Нижегородская обл.,2764,118.714286,123.928571,2918.428571,3258.642857,6,-115
9957,2020-08-13,22943,18803,358,7,162,78,0.932,1.032,2.527436,2.149132,Свердловская обл.,3782,174.000000,183.642857,3537.857143,3291.714286,-5,77
12127,2020-08-13,18219,15649,145,2,108,65,0.776,1.028,2.007033,1.378013,ХМАО – Югра,2425,124.285714,146.928571,2399.142857,2789.357143,-9,41
3096,2020-08-13,15101,10943,221,7,115,182,0.956,1.034,1.663549,2.237211,Иркутская обл.,3937,121.428571,129.928571,4066.571429,4320.142857,-4,-74
5115,2020-08-13,14812,12344,380,4,121,63,0.963,1.036,1.631712,1.186512,Красноярский край,2088,125.142857,129.428571,1897.428571,1628.285714,-2,54
8867,2020-08-13,14613,12002,274,3,125,122,0.984,1.037,1.609790,1.328007,Ростовская обл.,2337,127.142857,128.928571,2409.285714,2422.857143,-2,0
12592,2020-08-13,12433,7931,98,0,84,102,0.939,1.029,1.369638,2.502586,Челябинская обл.,4404,87.142857,96.642857,4389.857143,4412.428571,-4,-18


In [25]:



fig = px.pie(df2, values='Confirmed', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего выявленных случаев заболевания COVID-19 в России: {df2.Confirmed.sum()}<br>'+
            f'(на дату: {myday})'
            ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  margin = {'t':150, 'b':0},
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [26]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_ill'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_ill']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(20)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6521,2020-08-13,250303,193473,4622,11,692,787,1.007,1.011,27.573759,29.667345,Москва,52208,690.714286,688.500000,53014.714286,54670.785714,3,-106
6686,2020-08-13,65472,49066,1136,4,148,150,0.968,1.009,7.212495,8.677221,Московская обл.,15270,153.571429,155.214286,15348.714286,15462.642857,-4,-6
9337,2020-08-13,33525,24354,2203,15,160,96,1.006,1.019,3.693165,3.959586,Санкт-Петербург,6968,158.142857,158.785714,6763.000000,6507.714286,-1,49
12592,2020-08-13,12433,7931,98,0,84,102,0.939,1.029,1.369638,2.502586,Челябинская обл.,4404,87.142857,96.642857,4389.857143,4412.428571,-4,-18
9492,2020-08-13,10633,6614,64,0,95,93,1.013,1.038,1.171347,2.247440,Саратовская обл.,3955,95.000000,94.642857,3906.857143,3828.000000,-1,2
3096,2020-08-13,15101,10943,221,7,115,182,0.956,1.034,1.663549,2.237211,Иркутская обл.,3937,121.428571,129.928571,4066.571429,4320.142857,-4,-74
9957,2020-08-13,22943,18803,358,7,162,78,0.932,1.032,2.527436,2.149132,Свердловская обл.,3782,174.000000,183.642857,3537.857143,3291.714286,-5,77
1856,2020-08-13,10383,6669,75,0,102,91,1.000,1.039,1.143807,2.067872,Волгоградская обл.,3639,96.000000,95.071429,3605.000000,3607.428571,11,11
6841,2020-08-13,11656,8469,107,3,67,149,0.673,1.032,1.284043,1.750219,Мурманская обл.,3080,91.285714,111.642857,3280.000000,3540.071429,-5,-85
11972,2020-08-13,10440,7457,73,0,94,73,0.943,1.041,1.150086,1.653616,Ульяновская обл.,2910,100.714286,103.214286,2842.571429,2766.928571,3,21


In [27]:


fig = px.pie(df2, values='Remaining_ill', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
            f'(на дату: {myday})'
             ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':150, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [28]:
fig_r8 = go.Figure(data=[go.Pie(labels=df2.Region, values=df2.Remaining_ill, textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig_r8.update_layout(
                title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
                f'(на дату: {myday})',
                 margin = {'t':120, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  width = 900, height=900,
                  title_xanchor = "center",
                  title_yanchor = "bottom")
fig_r8.show()

In [29]:
covid_rates = pd.DataFrame()
mask = (X.Date>=datetime.date(2020,8,3))&(X.Date<=datetime.date(2020,8,10))&(X.Region.isin(regions_towatch))&(X.Region!='Россия')
                                      
box_cases1 = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum()
box_cases2 = X[mask].groupby(['Region','Date'])['Day_recovered'].sum()
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))

42 Москва
43 Московская обл.
56 Россия
60 Санкт-Петербург
61 Саратовская обл.
81 Челябинская обл.


In [30]:
new_df1=box_cases1.unstack().T
new_df1.tail(20)

Region,Москва,Московская обл.,Санкт-Петербург,Саратовская обл.,Челябинская обл.
Date,,,,,
2020-08-03,693,160,160,94,107
2020-08-04,691,161,158,93,105
2020-08-05,687,162,156,94,103
2020-08-06,684,159,159,90,92
2020-08-07,686,158,157,94,89
2020-08-08,691,161,155,95,87
2020-08-09,689,149,160,96,94
2020-08-10,694,153,157,95,83


In [31]:
color_dict = {
                "Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Иркутская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Челябинская обл.': 'indianred'}

cols = list(new_df1.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по новым заболевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Новые заболевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [32]:
new_df2=box_cases2.unstack().T
new_df2.tail(20)

Region,Москва,Московская обл.,Санкт-Петербург,Саратовская обл.,Челябинская обл.
Date,,,,,
2020-08-03,842,197,75,35,3
2020-08-04,1456,207,85,85,170
2020-08-05,1329,191,37,107,142
2020-08-06,1254,189,84,80,141
2020-08-07,1353,194,93,72,120
2020-08-08,1278,191,83,86,119
2020-08-09,803,176,33,20,29
2020-08-10,898,169,45,41,78


In [33]:

cols = list(new_df2.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по выздоровевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Выздоровевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [34]:
from plotly.subplots import make_subplots

# fig = tools.make_subplots(rows = 1, cols = 2, specs = [[{}, {}]],
#                           horizontal_spacing = 0.05)


fig = make_subplots(rows=2, cols=1,specs = [[{}],[{}]],vertical_spacing = 0.03,shared_xaxes=True,
                  subplot_titles=("Box-график распределения по новым заболевшим(в день) за неделю:",
                                 "Box-график распределения по выздоровевшим(в день) за неделю:"))
#subplot_titles=("Статистика по новым заболевшим за последние 2 недели","Статистика по выздоровевшим за последние 2 недели")

for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)),row=1,col=1)

for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)),row=2,col=1)


fig.update_layout(
#     annotations=[
#         dict(
#             x=0.5,
#             y=-0.15,
#             showarrow=False,
#             text="Ветикальная пунктирная линия - среднее, Вертикальная сплошная - медиана распределения",
#             xref="paper",
#             yref="paper"),
        
#                 ],
    
    
                  width = 990, height = 1200,
    
    
                 showlegend=False,template ='ggplot2'
)

#fig.update_layout(height=1000, width=800,showlegend=False)
fig.show()